In [ ]:
import arcpy
from arcpy.sa import *

# Set environment settings
arcpy.env.overwriteOutput = True
arcpy.env.workspace = r"C:\Projects\Vancouver Parks\Vancouver Parks GIS\Vancouver Parks GIS.gdb"
dtm_raster = r"C:\Projects\Vancouver Parks\Vancouver Parks GIS\Vancouver Parks GIS.gdb\DTM_v19_Clipped" 
output_folder = r"C:\Projects\Vancouver Parks\DTM\Flood Extent Processing"        

# Define flood elevation scenarios (in meters)
flood_levels = {
    "200yr_RP": 2.83,
    "200yr_RP_SLR_0_5m": 3.33,
    "200yr_RP_SLR_1m": 3.83,
    "200yr_RP_SLR_2m": 4.83
}

# Step 1: Define coastal seed (areas <= 0m as flood source)
coastal_seed = "FloodExtent_CoastalSeed"

# Process each flood scenario
for scenario, elevation in flood_levels.items():
    print(f"Processing flood extent for {scenario} at {elevation}m...")
    
    # Step 2: Reclassify DTM to binary raster (≤ elevation = 1, > elevation = NoData)
    flood_raster = Con(Raster(dtm_raster) <= elevation, 1)
    flood_raster.save(f"{output_folder}/{scenario}_flood.tif")
    
    # Step 3: Group contiguous regions
    region_groups = RegionGroup(flood_raster, "EIGHT", "WITHIN", "NO_LINK")
    region_groups.save(f"{output_folder}/{scenario}_regions.tif")
    
    # Step 4: Filter regions connected to the coast
    # Use Zonal Statistics to identify regions touching the coastal seed
    zonal_table = f"{output_folder}/{scenario}_zonal_table.dbf"
    ZonalStatisticsAsTable(region_groups, "Value", coastal_seed, zonal_table, "DATA", "SUM")
    
    # Select regions with SUM > 0 (connected to coast)
    connected_regions = Con(IsNull(region_groups), 0, 
                          Con(InList(region_groups, [row[0] for row in arcpy.da.SearchCursor(zonal_table, "Value", "SUM > 0")]), 1))
    connected_regions.save(f"{output_folder}/{scenario}_connected.tif")
    
    # Step 5: Convert to polygon
    output_polygon = f"{output_folder}/{scenario}_flood_extent.shp"
    arcpy.RasterToPolygon_conversion(connected_regions, output_polygon, "SIMPLIFY", "Value")
    
    # Clean up intermediate files (optional)
    # arcpy.Delete_management(f"{output_folder}/{scenario}_flood.tif")
    # arcpy.Delete_management(f"{output_folder}/{scenario}_regions.tif")
    # arcpy.Delete_management(zonal_table)

print("Flood extent polygons created for all scenarios.")

Processing flood extent for 200yr_RP at 2.83m...
Processing flood extent for 200yr_RP_SLR_0_5m at 3.33m...
Processing flood extent for 200yr_RP_SLR_1m at 3.83m...
